In [1]:
# 

In [2]:
import json
with open("../../rag_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

documents = splitter.create_documents(
    texts=[item["content"] for item in data],
    metadatas=[item["metadata"] for item in data]
)


In [4]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\d.fernandez.macias\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [6]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [7]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="resumes",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

C:\Users\d.fernandez.macias\AppData\Local\Temp\ipykernel_26240\439416039.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [8]:
# vectorize!
qdrant.upload_points(
    collection_name="resumes",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc.page_content).tolist(),
            payload={**doc.metadata, "text": doc.page_content}
        ) for idx, doc in enumerate(documents)
    ]
)

In [9]:
user_prompt = "I need someone to program in javascript"

In [ ]:
hits = qdrant.search(
    collection_name="resumes",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=50
)

In [11]:
from collections import defaultdict

file_stats = defaultdict(lambda: {"count": 0, "max_score": 0, "chunks": []})

for hit in hits:
    fname = hit.payload["file_name"]
    file_stats[fname]["count"] += 1
    file_stats[fname]["max_score"] = max(file_stats[fname]["max_score"], hit.score)
    file_stats[fname]["chunks"].append(hit.payload["text"])

# (count * max_score) to prioritize
sorted_files = sorted(
    file_stats.items(),
    key=lambda x: (x[1]["count"] * x[1]["max_score"]),
    reverse=True
)

# Step 3: Build a context string from the top 2 results
top_n = 2
context_chunks = []

for i, (fname, stats) in enumerate(sorted_files[:top_n]):
    joined_chunks = "\n".join(stats["chunks"])
    context_chunks.append(f"From {fname}:\n{joined_chunks}")

main_context = "\n\n---\n\n".join(context_chunks)

# Optional: get names of other people you *didn't* include
excluded = [fname for fname, _ in sorted_files[top_n:]]
also_mentioned = ", ".join(set(excluded))


In [ ]:
import ollama
o_models=["deepseek-r1:1.5b","phi4:latest"]
# Construct your RAG prompt
prompt_text = f"""
You are an assistant specialized in analyzing resumes. Based on the following resume information:

{main_context}

Answer the question: {user_prompt}
"""
# Call the model with the chat interface
response = ollama.chat(
    model=o_models[1],
    messages=[
        {"role": "system", "content": "You are a helpful assistant for analyzing CVs and finding relevant candidates."},
        {"role": "user", "content": prompt_text}
    ],

)

print(response['message']['content'])


I'm sorry, but as an AI text-based model, I don't have the capability to remember personal data or previous interactions unless they are included in the current session's input. If you mentioned your name earlier in this conversation and want me to use it again, please feel free to remind me of it!
